# Integrador - Teoria de control

In [1457]:
import sympy
from sympy.physics.control.lti import TransferFunction
import functools 
import copy

# Planteo del modelo
## Ecuaciones
Sean 

$\rarr$ x1: temperatura actual del sistema

$\rarr$ x2: Temperatura del resistor



In [1458]:
from sympy.abc import s # Laplace var 
a,b,c,d,e= sympy.symbols("a,b,c,d,e") # State space var 

A = sympy.Matrix([[a,b],[c,d]])
B = sympy.Matrix([[0],[e]])
C = sympy.Matrix([1,0]).T

### Matriz dinámica

In [1459]:
A

Matrix([
[a, b],
[c, d]])

### Matriz de control

In [1460]:
B

Matrix([
[0],
[e]])

### Matriz de salida

In [1461]:
C

Matrix([[1, 0]])

## Funcion de transferencia

El calculo de la funcion de transferencia se hace a través de la formula:


$$
 H(s) = C{(sI - A)^{-1}}B + D
$$


In [1462]:
temp = s * sympy.eye(A.shape[0]) - A # sI - A
inv = temp.inv()                  # (sI - A)^-1
temp = C * inv                  # C * (sI - A)^-1
H = temp * B                   # C * (sI - A)^-1 * B
transfer_function = H[0]      # Extract the result

num,den = sympy.fraction(sympy.factor(transfer_function)) # Factorize (factor) and extract numerator and denominator (fraction)
tf_expr = num/den   ##This is useful for jordan form

tf = TransferFunction(num,den,s)    # Create a transfer function object, which can be used for further analysis
tf

TransferFunction(b*e, a*d - a*s - b*c - d*s + s**2, s)

## Formas canonicas

Para el calculo de las formas canonicas, primero se preparan los datos para su facil manipulacion luego

In [1463]:
den_args_str = [str(x) for x in den.args] # Extract terms from denominator
pow2 = [x for x in den_args_str if "s**2" in x] # Terms that have s ^ 2
pow1 = [x for x in den_args_str if "*s" in x and x not in pow2] # Terms that have s ^ 1
tail_terms = [x for x in den_args_str if x not in pow1 and x not in pow2] # Terms without s

pow1_expr =sympy.factor(sympy.parse_expr(functools.reduce(lambda a,b: a + b, pow1), evaluate=False)) # Join the terms
tail_terms_expr = sympy.factor(sympy.parse_expr(functools.reduce(lambda a,b: a + b, tail_terms), evaluate=False)) # Join the terms


def sanitize_expr_str(e):
    """
    Delete the s^2 and s^1 from the expression
    """
    start = e.find("s")
    if start == -1:
        return e 
    return e.replace(e[start:start+2:1], "")

# Find the coefficients a_0, a_1 and b_1
a_0 = sympy.parse_expr(sanitize_expr_str(str(tail_terms_expr)), evaluate=False)
a_1 =sympy.parse_expr(sanitize_expr_str(str(pow1_expr)), evaluate=False)
b_0 = 0
b_1 = num

### Forma canonica de control

In [1464]:
A_ctrl = sympy.Matrix([[0,1], [a_0, a_1]])
B_ctrl = sympy.Matrix([[0], [1]])
C_ctrl = sympy.Matrix([b_1,b_0]).T

In [1465]:
A_ctrl

Matrix([
[        0,      1],
[a*d - b*c, -a - d]])

In [1466]:
B_ctrl

Matrix([
[0],
[1]])

In [1467]:
C_ctrl

Matrix([[b*e, 0]])

### Forma observable
Transponemos A e intercambiamos B y C de la forma canonica de control

In [1468]:
A_obs = copy.deepcopy(A_ctrl).T
B_obs = copy.deepcopy(C_ctrl)
C_obs = copy.deepcopy(B_ctrl).T

In [1469]:
A_obs

Matrix([
[0, a*d - b*c],
[1,    -a - d]])

In [1470]:
B_obs

Matrix([[b*e, 0]])

In [1471]:
C_obs

Matrix([[0, 1]])

### Forma de Jordan

In [1472]:
tf_poles = tf.poles()


In [1473]:
tf_poles[0]


a/2 + d/2 - sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2

In [1474]:
tf_poles[1]


a/2 + d/2 + sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2

Usando los polos encontrados:

In [1475]:
A_jordan = sympy.Matrix([[tf_poles[0], 0], [0,tf_poles[1]]])
A_jordan

Matrix([
[a/2 + d/2 - sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2,                                               0],
[                                              0, a/2 + d/2 + sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2]])

In [1476]:
B_jordan = sympy.ones(len(tf_poles),1)
B_jordan

Matrix([
[1],
[1]])

In [1477]:
C_jordan = sympy.Matrix([        # Residue calculates the residue of the expression for a given pole
    sympy.residue(tf_expr, s, tf_poles[0]), 
    sympy.residue(tf_expr, s, tf_poles[1])
    ] ).T
C_jordan

Matrix([[-b*e/sqrt(a**2 - 2*a*d + 4*b*c + d**2), b*e/sqrt(a**2 - 2*a*d + 4*b*c + d**2)]])

## Asignando valores a los coeficientes
En la presente seccion, aplicamos valores a los coeficientes de modo de instanciar el sistema y poder obtener los resultados requeridos

Si bien creemos que la generalidad mostrada anteriormente es lo mas imporante del estudio, entendemos que para realizar el analisis es importante instanciar los coeficientes. 

### Ecuaciones
Como se observara, se tiene en cuenta que la inferencia de la temperatura del resistor sobre la tasa de cambio de la misma es nula.

In [1478]:
a_value, b_value, c_value, d_value, e_value =-1,2,0,-2,1
# For simplicity, we replace the values in all matrices
A = A.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
B = B.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
C = C.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})

#### Matriz dinámica

In [1479]:
A

Matrix([
[-1,  2],
[ 0, -2]])

#### Matriz de control

In [1480]:
B

Matrix([
[0],
[1]])

#### Matriz de salida

In [1481]:
C

Matrix([[1, 0]])

### Funcion de transferencia

In [1482]:
tf = tf.evalf(subs={a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
tf

TransferFunction(2.0, s**2 + 3.0*s + 2.0, s)

In [1483]:
tf.poles()

[-2.00000000000000, -1.00000000000000]

### Formas canonicas

In [1484]:
a_0 = a_0.evalf(subs={a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
a_1 = a_1.evalf(subs={a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
b_1 = b_1.evalf(subs={a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
[a_0, a_1, b_1]

[2.00000000000000, 3.00000000000000, 2.00000000000000]

#### Forma canonica de control

In [1485]:
A_ctrl = A_ctrl.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
B_ctrl = B_ctrl.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
C_ctrl = C_ctrl.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})

In [1486]:
A_ctrl

Matrix([
[0, 1],
[2, 3]])

In [1487]:
B_ctrl

Matrix([
[0],
[1]])

In [1488]:
C_ctrl

Matrix([[2, 0]])

#### Forma canonica observable

In [1489]:
A_obs = A_obs.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
B_obs = B_obs.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
C_obs = C_obs.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})

In [1490]:
A_obs

Matrix([
[0, 2],
[1, 3]])

In [1491]:
B_obs

Matrix([[2, 0]])

In [1492]:
C_obs

Matrix([[0, 1]])

#### Forma canonica de Jordan

In [1493]:
A_jordan = A_jordan.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
B_jordan = B_jordan.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})
C_jordan = C_jordan.subs({a:a_value, b:b_value, c:c_value, d:d_value, e:e_value})

In [1494]:
A_jordan

Matrix([
[-2,  0],
[ 0, -1]])

In [1495]:
B_jordan

Matrix([
[1],
[1]])

In [1496]:
C_jordan

Matrix([[-2, 2]])